In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/final-clean/final_data.csv


In [35]:
p_df = pd.read_csv('../input/final-clean/final_data.csv')
docs = p_df['TEXT'].values

In [36]:
from tqdm import tqdm
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
def docs_preprocessor(docs):
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])
    return docs
docs = docs_preprocessor(docs)

In [37]:
# docs

In [38]:
%time
from gensim.models import Phrases
# Add bigrams and trigrams to docs (only ones that appear 10 times or more).
print("here1")
bigram = Phrases(docs, min_count=10)
print("here2")
trigram = Phrases(bigram[docs])
print("here3")

CPU times: user 0 ns, sys: 3 µs, total: 3 µs
Wall time: 7.39 µs
here1
here2
here3


In [39]:
for idx in tqdm(range(len(docs))):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

100%|██████████| 3321/3321 [02:28<00:00, 22.34it/s]


In [40]:
%%time
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=10, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 303146
Number of unique words after removing rare and common words: 100000
CPU times: user 35.3 s, sys: 87.9 ms, total: 35.4 s
Wall time: 35.4 s


In [41]:
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [42]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 100000
Number of documents: 3321


In [43]:
%%time
from gensim.models import LdaModel
# Set training parameters.
num_topics = 9
chunksize = 500 # size of the doc looked at every pass
passes = 20 # number of passes through documents
iterations = 400
eval_every = 1  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

CPU times: user 51min 2s, sys: 49min 8s, total: 1h 40min 11s
Wall time: 26min 14s
CPU times: user 51min 2s, sys: 49min 8s, total: 1h 40min 11s
Wall time: 26min 14s


In [49]:
%%time
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
# feed the LDA model into the pyLDAvis instance
gensimvis.prepare(model, corpus, dictionary)

CPU times: user 39.1 s, sys: 21.4 s, total: 1min
Wall time: 30.4 s


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.147196  0.009527       1        1  32.536748
6      0.153060 -0.119124       2        1  11.881623
1     -0.127529 -0.214663       3        1  10.873769
3      0.110297  0.185947       4        1   9.183398
2     -0.256311  0.145982       5        1   7.931280
4      0.130157  0.056538       6        1   7.886175
5     -0.196790 -0.191721       7        1   7.852503
8      0.157739 -0.086014       8        1   6.354134
7     -0.117818  0.213528       9        1   5.500370, topic_info=             Term          Freq         Total Category  logprob  loglift
10094         alk  31171.000000  31171.000000  Default  30.0000  30.0000
545           raf  34688.000000  34688.000000  Default  29.0000  29.0000
187     cyclin_d1  25336.000000  25336.000000  Default  28.0000  28.0000
10473       k_ras  26821.000000  26821.000000  Default  27.0000  27.0000
29143       b_raf  25971.000000  25971.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
33900   lapatinib   2136.446832   3294.167408   Topic9  -5.8970   2.4673
3498        ba_f3   4378.245200  19861.373539   Topic9  -5.1795   1.3882
19653         msi   1876.637125   3241.992962   Topic9  -6.0267   2.3536
3873   ic50_value   1987.994126   5695.559160   Topic9  -5.9691   1.8478
10554    mismatch   1662.172649   2301.270236   Topic9  -6.1481   2.5750

[500 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
7352       1  0.070873           00
7352       4  0.050087           00
7352       5  0.873268           00
7352       7  0.003757           00
7352       8  0.001753           00
...      ...       ...          ...
9295       8  0.011622    β_catenin
13881      6  0.999575  γ_phosphate
53001      2  0.999819  γ_secretase
93173      7  0.999460       δb_raf
9297       2  0.999768           κb

[916 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 2, 4, 3, 5, 6, 9, 8])